In [1]:
# pip install accelerate
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
import requests
import torch

model_id = "google/medgemma-4b-it"

model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)



/home/d1l1th1um/Desktop/demo/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Fetching 2 files: 100%|██████████| 2/2 [00:00<00:00, 15768.06it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:

image = Image.open("image.jpg")

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert radiologist."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "This is my lung. Do you think that i'm healthy?"},
            {"type": "image", "image": image}
        ]
    }
]

inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=500, do_sample=False)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

Okay, I've reviewed the chest X-ray you provided.

**Overall Impression:**

The chest X-ray appears relatively normal. There are no obvious signs of acute lung pathology such as pneumonia, pneumothorax, or large pleural effusions. The heart size appears within normal limits. The mediastinum is also unremarkable.

**Specific Observations:**

*   **Lung Fields:** The lung fields are clear bilaterally, with no focal consolidations, masses, or infiltrates identified.
*   **Heart Size:** The heart size appears within normal limits.
*   **Mediastinum:** The mediastinum is unremarkable.
*   **Bones:** The visualized bony structures (ribs, clavicles, and thoracic spine) appear intact.
*   **Diaphragm:** The diaphragms are well-defined.
*   **Mediastinal Structures:** The mediastinal structures, including the trachea and major vessels, appear normal.

**Possible Considerations:**

*   **Subtle Findings:** It's always possible that very subtle findings could be present that are not readily appar